In [17]:
import numpy as np
import sympy as sp


def beale_function(x, y):
    #f(x_1,x_2) = (1.5 - x_1 + x_1 x_2)^2 + (2.25 - x_1 + x_1 x_2^2)^2 + (2.625 - x_1 + x_1 x_2^3)^2
    return (1.5 - x + x*y)**2 + (2.25 - x + x*y**2)**2 + (2.625 - x + x*y**3)**2

In [18]:
class NiceFunction:
    def __init__(self, f):
        self.f = f
        self.x = sp.Symbol('x')
        self.y = sp.Symbol('y')
        self.f_x = sp.diff(f(self.x,self.y), self.x)
        self.f_y = sp.diff(f(self.x,self.y), self.y)
        self.gradient_f_sym = np.array([self.f_x, self.f_y])
        self.f_xx = sp.diff(self.f_x, self.x)
        self.f_yy = sp.diff(self.f_y, self.y)
        self.f_xy = sp.diff(self.f_x, self.y)
        self.f_yx = sp.diff(self.f_y, self.x)
        self.hessian_f_sym = np.array([[self.f_xx, self.f_xy], [self.f_yx, self.f_yy]])
    def gradient_f(self, val_x, val_y):
        #thay số vào đạo hàm
        ans = np.array([0,0])
        for i in range(2):
                ans[i] += self.gradient_f_sym[i].subs([(self.x, val_x), (self.y, val_y)])
        return ans

    def hessian_f(self, val_x, val_y):
        ans = np.array([[0,0],[0,0]])
        for i in range(2):
            for j in range(2):
                ans[i][j] = self.hessian_f_sym[i][j].subs([(self.x, val_x), (self.y, val_y)])
                 
        

In [19]:
niceFunction = NiceFunction(beale_function)
print(niceFunction.f)
print(niceFunction.gradient_f_sym)
print(niceFunction.hessian_f_sym)


<function beale_function at 0x000001519D5FC220>
[2.25*(1.33333333333333*y - 1.33333333333333)*(0.666666666666667*x*y - 0.666666666666667*x + 1) + 5.0625*(0.888888888888889*y**2 - 0.888888888888889)*(0.444444444444444*x*y**2 - 0.444444444444444*x + 1) + 6.890625*(0.761904761904762*y**3 - 0.761904761904762)*(0.380952380952381*x*y**3 - 0.380952380952381*x + 1)
 15.75*x*y**2*(0.380952380952381*x*y**3 - 0.380952380952381*x + 1) + 9.0*x*y*(0.444444444444444*x*y**2 - 0.444444444444444*x + 1) + 3.0*x*(0.666666666666667*x*y - 0.666666666666667*x + 1)]
[[2.25*(0.666666666666667*y - 0.666666666666667)*(1.33333333333333*y - 1.33333333333333) + 5.0625*(0.444444444444444*y**2 - 0.444444444444444)*(0.888888888888889*y**2 - 0.888888888888889) + 6.890625*(0.380952380952381*y**3 - 0.380952380952381)*(0.761904761904762*y**3 - 0.761904761904762)
  7.875*x*y**2*(0.761904761904762*y**3 - 0.761904761904762) + 4.5*x*y*(0.888888888888889*y**2 - 0.888888888888889) + 2.0*x*y + 1.5*x*(1.33333333333333*y - 1.33333

In [20]:
def trial_function(x,y):
    return x**3 + y**3 + x*y
nice_trial_function = NiceFunction(trial_function)
print(nice_trial_function.f)
print(nice_trial_function.gradient_f_sym)
print(nice_trial_function.hessian_f_sym)
#thay số vào hàm trong sympy


<function trial_function at 0x00000151A6B71940>
[3*x**2 + y x + 3*y**2]
[[6*x 1]
 [1 6*y]]


In [21]:
class NewtonFindMin:
    def __init__(self, niceFunction, x0, epsilon=1e-4, max_iterations=1000, alpha = 1,
                 learning_rate = 1):
        self.niceFunction = niceFunction
        self.x0 = x0
        self.epsilon = epsilon
        self.max_iterations = max_iterations
        self.alpha = alpha
        self.learning_rate = learning_rate
    def gradient_descent_backtracking_linesearch(self, contraction_factor=0.5):
        x = self.x0[0]
        y = self.x0[1]
        for i in range(self.max_iterations):
            current_gradient = self.niceFunction.gradient_f(x, y)
            t = self.learning_rate
            if np.linalg.norm(current_gradient) < self.epsilon:
                break
            for j in range(self.max_iterations):
                next_x = x - t * current_gradient[0]
                next_y = y - t* current_gradient[1]
                while self.niceFunction.f(next_x, next_y) > self.niceFunction.f(x, y) - self.alpha *t* np.linalg.norm(current_gradient)**2 :
                    t = t*contraction_factor
                x = next_x
                y = next_y
        return x, y
    def newton_backtracking_linesearch(self, contraction_factor=0.5):
        x = self.x0[0]
        y = self.x0[1]
        for i in range(self.max_iterations):
            current_gradient = self.niceFunction.gradient_f(x, y)
            t = self.learning_rate
            if np.linalg.norm(current_gradient) < self.epsilon:
                break
            current_hess = self.niceFunction.hessian_f(x, y)
            v = np.linalg.solve(current_hess, current_gradient)
            for j in range(self.max_iterations):
                next_x = x - t * v[0]
                next_y = y - t* v[1]
                if self.niceFunction.f(next_x, next_y) > self.niceFunction.f(x, y) + self.alpha *t* current_gradient @ v :
                    t = t*contraction_factor
                else:
                    x = next_x
                    y = next_y
                    break
        return x, y 
        

In [22]:
nice_beale_function = NiceFunction(beale_function)
A = np.array([0,0])
B = np.array([-4,3])
newtonSolver = NewtonFindMin(nice_beale_function, A)
sol1 = newtonSolver.gradient_descent_backtracking_linesearch()
print(sol1)
beale_function(sol1[0], sol1[1])


KeyboardInterrupt: 

In [ ]:
newtonSolverB = NewtonFindMin(nice_beale_function, B)
sol2 = newtonSolverB.gradient_descent_backtracking_linesearch()
print(sol2)
beale_function(sol2[0], sol2[1])


C:\Users\ThinkPad\AppData\Local\Temp\ipykernel_7040\4023328981.py:7: RuntimeWarning: overflow encountered in scalar multiply
  return (1.5 - x + x*y)**2 + (2.25 - x + x*y**2)**2 + (2.625 - x + x*y**3)**2


(-3.9942551183048636, 2.9766560586285777)


10637.581838766582

In [ ]:
sol3 = newtonSolver.newton_backtracking_linesearch()
print(sol3)

LinAlgError: 0-dimensional array given. Array must be at least two-dimensional

In [ ]:
val_x = sp.Symbol('x')
y = sp.Symbol('y')
f = val_x+y
#thay số vào hàm trong sympy
f.subs(val_x, 1).subs(y, 2)

3